In [50]:
import numpy as np
import logging
import gzip
import matplotlib.pyplot as plt

PWD="/home/duzicman/projects/tau/statistical_machine_learning/logistic_classifier/datasets"
DATASET = {"labels": {"path" :PWD + "/train-labels-idx1-ubyte.gz", "offset":8}, "data": {"path": PWD + "/train-images-idx3-ubyte.gz", "offset":16}}
IMAGE_LENGTH = 28
IMAGE_SIZE = IMAGE_LENGTH ** 2


In [51]:
def load_dataset(file_path, offset):
    with gzip.open(file_path, 'rb') as data_file:
        data = np.frombuffer(data_file.read(), np.uint8, offset=offset)
    return data

features = load_dataset(DATASET["data"]["path"], DATASET["data"]["offset"])
labels = load_dataset(DATASET["labels"]["path"], DATASET["labels"]["offset"])


In [53]:
def one_hot_encoding(labels):
    label_ammount = np.max(labels) + 1
    return np.eye(label_ammount)[labels]

def format_dataset(features, labels):
    labels = one_hot_encoding(labels)
    features = np.reshape(features, (-1, IMAGE_SIZE))
    features = np.c_[features, np.ones(features.shape[0])]
    return features, labels

features, labels = format_dataset(features, labels)
class_number = labels.shape[1]


In [ ]:
def show_image(image):
    pixels = np.reshape(image, (IMAGE_LENGTH, IMAGE_LENGTH))
    plt.imshow(pixels, cmap='gray')
    plt.show()
# show_image(features[0])
#TODO I added extra normilization pixel so it doesnt work noe
# print(labels[0])


ValueError: cannot reshape array of size 785 into shape (28,28)